In [2]:
#KOSPI200 선물 이론가 산출
#엑셀에서 데이터 읽어오기
import pandas as pd
import numpy as np
#엑셀파일에서 데이터를 읽어올 때는 pd.read_excel()을 사용, pd.read_excel(액셀 파일명, 시트명, index_col=칼럼명), 시트명 생략시 1번시트, 칼럼명 생략시 0,1,2 로 인덱싱
#엑셀 파일이 파이썬 노트북 파일과 다른 폴더에 있다면 경로명 붙이기
df = pd.read_excel('09.2 KOSPI200 F 201712.xlsx', 'Sheet1', index_col = '일자') #같은 파일위치에 없으면 경로명 파이앞에 붙여야됨
df.head()
# 엑셀 데이터를 저장할 때는 데이터프레임.to_excel(파일명.xlsx, 시트명, indext = True) 이용,index=False로 바꾸면 인덱스는 저장X
# csv 파일은 df = pd.read_csv("abc.csv", encoding = " ")로 불러오기, 이 때 자동으로 맨 첫번째 행이 header(레이블)이 되므로
# 만약 레이블이 없을 시 encoding = " ", header = None, name = [ 레이블 ] 이런 식으로 하거나 name = [레이블] 적지 말고
# df.columns = [ 레이블 ] 로 따로 추가해주면 됨
# txt파일도 read_csv("abc.txt", sep = "\t") 로 읽어주면 됨

,선물가격,지수
일자,,
2017/09/01,308.40,307.78
2017/09/04,305.60,304.19
2017/09/05,304.85,304.06
2017/09/06,304.30,303.18
2017/09/07,308.10,306.97


In [3]:
#불러온 데이터를 정상적으로 이용하려면 인덱스칼럼이 날짜 형식이어야 함
type(df.index) #인덱스는 문자형식임을 확인

pandas.core.indexes.base.Index

In [4]:
df.index = pd.to_datetime(df.index) #인덱스를 날짜로 변환
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [5]:
#선물 이론가 산출 : F = Se^(r-d)*t, t는 잔존만기로 1년 남았으면 1로 계산, 따라서 t = (최종거래일-기준일)/252
#파이썬에서 두 날짜 사이에 며칠이 있는지 계산할 때는 (날짜2- 날짜1).days 이용 -> 이 경우는 주말이 포함
#공휴일을 제외한 영업일만을 계산하려면 np.busday_count(날짜1, 날짜2)를 이용, 이를 이용하면 t = np.busday_count(t0.date(), T.date()), t0는 기준일자, T는 만기일자
#자연상수 e는 np.exp()를 이용, e^(r-d)*t = np.exp((r-d)*t)
#이 내용을 바탕으로 선무 이론가를 산출하는 함수를 만들어보면
def future_price(S, r ,d, T, t0):
    t = np.busday_count(t0.date(), T.date()) / 252
    F = S * np.exp((r-d)*t)
    return F

In [7]:
#위 함수를 이용해 KOSPI200 선물 이론가를 계산해보면
T = pd.to_datetime('2017-12-14') #최종거래일
r = 0.025 #이자율
d = 0.015 #배당률
for n in (df.index):
    t0= n
    S = df.loc[n, '지수']
    F = future_price(S, r, d, T, t0)
    f ='{:,.2f}'.format(F) #이론가 숫자 포매팅, 정수부분은 천의 자리마다 쉼표를 추가하고 소수점 아래는 2자리까지만 표시하도록 하는 것
    df.loc[n, '이론가1'] = f #'이론가1' 칼럼에 값 추가
    


In [8]:
#보급형 공식을 이용한 선물 이론가 계산
#단리 개념의 공식 F = S[1+(r-d)*t/365] 이용

def S_future_price(S, r, d, T, T0):
    t = (T-t0).days / 365
    F = S* (1 +(r-d) *t)
    return F


In [10]:
T = pd.to_datetime('2017-12-14') #최종거래일
r = 0.0225 #이자율
d = 0.015 #배당률
for n in (df.index):
    t0 = n
    S = df.loc[n, '지수']
    F = S_future_price(S, r, d, T, t0)
    f = '{:,.2f}'.format(F) #,랑 . 띄어쓰기 하면 안됨
    df.loc[n, '이론가2'] = f


In [11]:
df

,선물가격,지수,이론가1,이론가2
일자,,,,
2017-09-01,308.40,307.78,308.69,308.44
2017-09-04,305.60,304.19,305.07,304.82
2017-09-05,304.85,304.06,304.93,304.68
2017-09-06,304.30,303.18,304.04,303.80
2017-09-07,308.10,306.97,307.82,307.59
...,...,...,...,...
2017-12-08,324.75,324.68,324.73,324.72
2017-12-11,324.85,325.09,325.13,325.11
2017-12-12,323.80,324.07,324.10,324.08


In [13]:
#WTI 선물 가격을 이용한 롤오버 비용 계산
#롤오버 비용이 손익에 미치는 영향이 얼마나 큰지 서부 텍사스 중질유(WTI) 선물가격을 바탕으로 모델링
#2010년부터 2017년까지 WTI 현물에 투자했을 경우와 선물에 투자했을 경우의 손익이 어떻게 될지 비교
#데이터는 블룸버그 출처, 참고로 블룸버그는 월사용료만 수백만원이 넘는 고가의 정보매체(대부분 금융회사가 국내외 금융정보조회를 위해 사용)
#최근월물(front monthm, lead month, first month), 차근월물(second month) 용어 유의

#엑셀에서 WTI 현물, 최근월물, 차근월물 데이터 읽어오기
import pandas as pd
import numpy as np
df1 = pd.read_excel('11.2 WTI.xlsx', 'Spot', index_col = 'Date')
df2 = pd.read_excel('11.2 WTI.xlsx', 'Front', index_col = 'Date')
df3 = pd.read_excel('11.2 WTI.xlsx', 'Second', index_col = 'Date')

In [14]:
df = pd.concat([df1, df2, df3], axis=1)
df

,Spot,Front,Second
Date,,,
2017-12-28,59.84,59.84,59.87
2017-12-27,59.64,59.64,59.69
2017-12-26,59.97,59.97,60.00
2017-12-22,58.42,58.47,58.54
2017-12-21,58.33,58.36,58.40
...,...,...,...
2010-01-08,82.75,82.75,83.30
2010-01-07,82.66,82.66,83.19
2010-01-06,83.18,83.18,83.75


In [15]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [16]:
#엑셀에서 WTI 만기일 정보 읽어오기
#롤오버는 유동성(필요한 만큼 차근월물을 확보할 수 있는지 여부)과 가격(유동성이 높아져 차근월물 가격이 합리적인지 여부)을 고려해 만기일과 가까운 일에 시행
#2010년부터 2017년까지 만기일에 롤오버를 시행하기 위해 만기일 정보가 필요
df_e = pd.read_excel('11.2 WTI.xlsx', 'Expiry')
df_e[120:125]

,Ticker,Month,Last Trade,Last Price
120,CLG10,2017-02-10,01/20/10,77.62
121,CLH10,2017-03-10,02/22/10,80.16
122,CLJ10,2017-04-10,03/22/10,81.25
123,CLK10,2017-05-10,04/20/10,83.45
124,CLM10,2017-06-10,05/20/10,68.01


In [17]:
#이 중 만기일은 Last Trade란에 들어있음
df_e['Last Trade'].head() #type이 날짜형식이 아님 -> 날짜 형식으로 바꿔야함

0    01/20/00
1    02/22/00
2    03/21/00
3    04/19/00
4    05/22/00
Name: Last Trade, dtype: object

In [19]:
expiry = pd.Series(pd.to_datetime(df_e['Last Trade']))
expiry.head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [20]:
#시세 데이터프레임에 최종거래일 표시하기
#최근월물을 팔고 차근월물을 사는 롤오버는 최종거래일에 일어나기 때문에 시세 데이터에 최종거래일을 표시해주어야 함
df['Expiry'] = np.nan #일단 빈 값으로 배정
df.head()

,Spot,Front,Second,Expiry
Date,,,,
2017-12-28,59.84,59.84,59.87,NaN
2017-12-27,59.64,59.64,59.69,NaN
2017-12-26,59.97,59.97,60.00,NaN
2017-12-22,58.42,58.47,58.54,NaN
2017-12-21,58.33,58.36,58.40,NaN


In [21]:
#Trading day중 만기일을 찾아서 표시하기
for d in df.index: #Date값을 하나씩 읽어와서
    if d in pd.to_datetime(expiry.values): #expiry에 같은 값이 있으면
        df.loc[d].Expiry = 1 #Expiry 열에 1이라고 표시
df[5:10] #df 테이블 일부를 표시해서 확인

,Spot,Front,Second,Expiry
Date,,,,
2017-12-20,58.02,58.09,58.13,NaN
2017-12-19,57.46,57.46,57.56,1.0
2017-12-18,57.16,57.16,57.22,NaN
2017-12-15,57.30,57.30,57.33,NaN
2017-12-14,57.04,57.04,57.08,NaN


In [23]:
#손익계산 : 현물에 투자했을 경우
#현물에 투자했다면 손익 = 현재가격 - 초기가격
price_now = float(df['2017-12-28']['Spot']) #현물 현재가격
price_before = float(df['2010-01-04']['Spot']) #현물 최초가격
profit = (price_now - price_before) * 1000
print('손익(현재가격 - 초기가격) :', '{:,.2f}'.format(profit))

손익(현재가격 - 초기가격) : -21,670.00


In [24]:
#손익계산 : 선물에 투자했을 경우
#현재가격 = 초기가격 - 롤오버비용
roll_over_cost_acc = 0 #롤오버 비용 누적값 초기화
for d in df.index: #df 일자별로 순환하며
    if (df.loc[d].Expiry ==1): #최종거래일이면 롤오버 계산 시작
        #롤오버비용 = 차근월물 가격 - 최근월물 가격
        roll_over_cost = df.loc[d].Second - df.loc[d].Front #롤오버비용
        roll_over_cost_acc = roll_over_cost_acc + roll_over_cost #누적값

print('누적 롤오버 비용:', '{:,.2f}'.format(roll_over_cost_acc))

누적 롤오버 비용: 40.68


In [26]:
#롤오버 비용 누적값을 반영해서 WTI 선물 손익을 계산하면
price_now = float(df['2017-12-28']['Front']) #선물 현재가
price_before = float(df['2010-01-04']['Front']) #선물 최초가
profit = (price_now - price_before - roll_over_cost_acc) * 1000
print('누적 손익', '{:,.2f}'.format(profit))

누적 손익 -62,350.00
